In [39]:
#import dependencies
import pandas as pd
import numpy as np
import os
import gmaps
import json
import requests
from pprint import pprint
import us

#import API key
from config import gkey, weather_api_key

In [68]:
#read the sunshine csv file into a dataframe
sunshine_df = pd.read_csv(os.path.join("..","potential_data_sets","NOAA_Sunshine_AveragePercentPossible.csv"))
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90%,42
1,REDDING,CA,88%,10
2,LAS VEGAS,NV,85%,47
3,PHOENIX,AZ,85%,101
4,TUCSON,AZ,85%,53
...,...,...,...,...
169,"POHNPEI, MICRONESIA",PC,41%,44
170,ELKINS,WV,40%,11
171,MT. WASHINGTON,NH,33%,64
172,QUILLAYUTE,WA,33%,30


In [69]:
#check the datatypes
sunshine_df.dtypes

CITY                              object
STATE                             object
ANNUAL % AVG POSSIBLE SUNSHINE    object
NO. OF YEARS OF DATA               int64
dtype: object

In [70]:
#strip the % column of the % symbol and convert to float
sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"] = sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"].map(lambda x: x.rstrip('%'))
sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"] = sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"].astype(float)
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90.0,42
1,REDDING,CA,88.0,10
2,LAS VEGAS,NV,85.0,47
3,PHOENIX,AZ,85.0,101
4,TUCSON,AZ,85.0,53
...,...,...,...,...
169,"POHNPEI, MICRONESIA",PC,41.0,44
170,ELKINS,WV,40.0,11
171,MT. WASHINGTON,NH,33.0,64
172,QUILLAYUTE,WA,33.0,30


In [71]:
#credit to: https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
#dictionary of state abbreviations:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [72]:
# #convert state to full state name

#displays 'AZ'
# sunshine_df['STATE'][0]

#errors out saying: module 'us.states' has no attribute 'sunshine_df'
# us.states.sunshine_df['STATE'][0].name

#displays 'Arizona'
# us.states.AZ.name

In [83]:
#JB's help/idea for a solution

#still didn't work, returning "module 'us.states' has no attribute 'short'" error
# shorts = sunshine_df['STATE']
# full_names = []
# for short in shorts:
#     full_name = us.states.short.name
#     full_names.append(full_name)

# full_names

In [73]:
#mannually clean the data set - we don't know how to web scrape yet
stateNames_df = pd.read_csv(os.path.join("..","potential_data_sets","50_us_states_all_data.csv"))
stateNames_df.head()

,CAPS STATE NAME,TITLE STATE NAME,2 LETTER ABBREVIATION,ABBREV.
0,ALABAMA,Alabama,AL,Ala.
1,ALASKA,Alaska,AK,Alaska
2,ARIZONA,Arizona,AZ,Ariz.
3,ARKANSAS,Arkansas,AR,Ark.
4,CALIFORNIA,California,CA,Calif.


In [74]:
#join the two dataframes
cleanSunshine_df = pd.merge(sunshine_df,stateNames_df,how='left',left_on="STATE",right_on="2 LETTER ABBREVIATION")
#clean up formatting
del cleanSunshine_df['STATE']
del cleanSunshine_df['CAPS STATE NAME']
del cleanSunshine_df['2 LETTER ABBREVIATION']
del cleanSunshine_df['ABBREV.']
cleanSunshine_df = cleanSunshine_df.rename(columns={"TITLE STATE NAME":"STATE"})
cleanSunshine_df = cleanSunshine_df.dropna().reset_index(drop=True)
cleanSunshine_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE
0,YUMA,90.0,42,Arizona
1,REDDING,88.0,10,California
2,LAS VEGAS,85.0,47,Nevada
3,PHOENIX,85.0,101,Arizona
4,TUCSON,85.0,53,Arizona
...,...,...,...,...
156,HILO,41.0,52,Hawaii
157,ELKINS,40.0,11,West Virginia
158,MT. WASHINGTON,33.0,64,New Hampshire
159,QUILLAYUTE,33.0,30,Washington


In [76]:
#weather api information
# api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API key}
url_currentWeather = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

#find the latitude and longitude of the cities
lat = []
lng = []

#loop through the cities_df to retrieve the weather information
for index, row in cleanSunshine_df.iterrows():
    currentCity = row["CITY"]
    currentState = row["STATE"]
    query_url = f"{url_currentWeather}q={currentCity},{currentState}&units={units}&appid={weather_api_key}"
    response = requests.get(query_url).json()
    
    try:
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        print(f"Processeing Record | {currentCity}\n{query_url}")
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print(f"City not found. Skipping {currentCity}...\n{query_url}")

#update latitude and longitude to actually match the city
cleanSunshine_df["Latitude"] = lat
cleanSunshine_df["Longitude"] = lng

Processeing Record | YUMA
http://api.openweathermap.org/data/2.5/weather?q=YUMA,Arizona&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | REDDING
http://api.openweathermap.org/data/2.5/weather?q=REDDING,California&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | LAS VEGAS
http://api.openweathermap.org/data/2.5/weather?q=LAS VEGAS,Nevada&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | PHOENIX
http://api.openweathermap.org/data/2.5/weather?q=PHOENIX,Arizona&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | TUCSON
http://api.openweathermap.org/data/2.5/weather?q=TUCSON,Arizona&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | EL PASO
http://api.openweathermap.org/data/2.5/weather?q=EL PASO,Texas&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | FRESNO
http://api.openweathermap.org/data/2.5/weather?q=FRESNO,California&units=imperial&appid=

Processeing Record | SHERIDAN
http://api.openweathermap.org/data/2.5/weather?q=SHERIDAN,Wyoming&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | WILMINGTON
http://api.openweathermap.org/data/2.5/weather?q=WILMINGTON,North Carolina&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | CHARLOTTE
http://api.openweathermap.org/data/2.5/weather?q=CHARLOTTE,North Carolina&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | GALVESTON
http://api.openweathermap.org/data/2.5/weather?q=GALVESTON,Texas&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | LITTLE ROCK
http://api.openweathermap.org/data/2.5/weather?q=LITTLE ROCK,Arkansas&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SAVANNAH
http://api.openweathermap.org/data/2.5/weather?q=SAVANNAH,Georgia&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | TULSA
http://api.openweathermap.org/data/2.5/weather

Processeing Record | FARGO
http://api.openweathermap.org/data/2.5/weather?q=FARGO,North Dakota&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
City not found. Skipping MIDDLETOWN/HARRISBURG AP...
http://api.openweathermap.org/data/2.5/weather?q=MIDDLETOWN/HARRISBURG AP,Pennsylvania&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | NEW ORLEANS
http://api.openweathermap.org/data/2.5/weather?q=NEW ORLEANS,Louisiana&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | PORTLAND
http://api.openweathermap.org/data/2.5/weather?q=PORTLAND,Maine&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | ST. LOUIS
http://api.openweathermap.org/data/2.5/weather?q=ST. LOUIS,Missouri&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
City not found. Skipping ATLANTIC CITY AP...
http://api.openweathermap.org/data/2.5/weather?q=ATLANTIC CITY AP,New Jersey&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | C

Processeing Record | JUNEAU
http://api.openweathermap.org/data/2.5/weather?q=JUNEAU,Alaska&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14


In [77]:
cleanSunshine_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE,Latitude,Longitude
0,YUMA,90.0,42,Arizona,32.73,-114.62
1,REDDING,88.0,10,California,40.59,-122.39
2,LAS VEGAS,85.0,47,Nevada,36.17,-115.14
3,PHOENIX,85.0,101,Arizona,33.45,-112.07
4,TUCSON,85.0,53,Arizona,32.22,-110.93
...,...,...,...,...,...,...
156,HILO,41.0,52,Hawaii,19.73,-155.09
157,ELKINS,40.0,11,West Virginia,38.93,-79.85
158,MT. WASHINGTON,33.0,64,New Hampshire,NaN,NaN
159,QUILLAYUTE,33.0,30,Washington,NaN,NaN


In [80]:
#clean the data set of null values
sunshineLocations_df = cleanSunshine_df.dropna().reset_index(drop=True)
sunshineLocations_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE,Latitude,Longitude
0,YUMA,90.0,42,Arizona,32.73,-114.62
1,REDDING,88.0,10,California,40.59,-122.39
2,LAS VEGAS,85.0,47,Nevada,36.17,-115.14
3,PHOENIX,85.0,101,Arizona,33.45,-112.07
4,TUCSON,85.0,53,Arizona,32.22,-110.93
...,...,...,...,...,...,...
139,NOME,42.0,40,Alaska,64.50,-165.41
140,ANCHORAGE,41.0,40,Alaska,61.22,-149.90
141,HILO,41.0,52,Hawaii,19.73,-155.09
142,ELKINS,40.0,11,West Virginia,38.93,-79.85


In [27]:
#add the heat layer to the gmap
